|   | name           | matrikelnumber |
|---|----------------|----------------|
| 1 | Mond Ferdinand | 11945397       |
| 2 | Poll Thomas    | 01629100       |

!!! For the animation to work i had to install ffmpeg by using the command 'conda install -c conda-forge ffmpeg' !!!


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline

In [2]:
def draw_animation(pulse='square', courant=1):
    U = 1 # convection velocity
    dx = 0.01 # step along x
    dt = courant * dx/U
    print('Timestep dt:', dt)
    #dt = 0.01 # timestep
    t_final = 3 # simulation time
    x_max = 1.5 # x range

    x = np.arange(0, x_max+dx, dx)
    t = np.arange(0, t_final+dt, dt)


    # calculate courant number




    # define initial conditions
    # square
    def init_square(x):
        result = 0
        if x > 0.1 and x < 0.3:
            result = 1
        return result

    square = np.vectorize(init_square)


    # gaussian
    gauss = lambda x: np.exp(-10*(4*x-1)**2)


    # define and simulate the evolution of the initial condition
    grid_1 = np.zeros([len(t), len(x)])

    if pulse == 'square':
        grid_1[0, :] = square(x)
    else:
        grid_1[0, :] = gauss(x)
        
        
    for i_t in range(len(t)-1):
        for i_x in range(len(x)):
            # Upwind scheme
            grid_1[i_t + 1, i_x] = grid_1[i_t, i_x] - courant * (grid_1[i_t, i_x] - grid_1[i_t, i_x-1])


    # plot simulation
    fig, ax = plt.subplots(figsize=(12,8))
    line_1, = ax.plot(x, np.zeros(len(x)))
    ax.set_ylim(-0.1, 1.1)


    def animate(i, grid_1):
        line_1.set_ydata(grid_1[i+1, :])  # update the data.

        return line_1


    text = 'gaussian pulse, courant = {}'.format(courant)

    ani = animation.FuncAnimation(
        fig, animate, fargs=(grid_1,), frames=len(t)-1, interval=80, blit=False)


    plt.title(text)
    plt.xlabel('C')
    plt.ylabel('x')
    plt.grid()
    plt.close(ani._fig)

    return HTML(ani.to_html5_video())

In [3]:
draw_animation(pulse='square', courant=1)

Timestep dt: 0.01


 When we simulate the evolution of the signal with a Courant number of 1, we get a stable simulation and no artificial diffusion which could be introduced due to the truncation error. <br><br>
The stability analysis states that the courant number has to be <= 1 for a stabel simulation. <br><br>
The truncation error is dependent on Constant * (1 - Courant). It follows that the error has to be 0 for a courant number of 1.

In [4]:
draw_animation(pulse='square', courant=0.95)

Timestep dt: 0.0095


When we use a courant number smaller than 1 we introduce an aritficial diffusifity which smoothes out the corners of the square pulse. <br><br>
The amplitude of the signal goes down and the pulse gets a gaussian shape.

In [5]:
draw_animation(pulse='square', courant=1.05)

Timestep dt: 0.0105


With a courant number of >1 we observe an instable simulation.

In [6]:
draw_animation(pulse='gauss', courant=1)

Timestep dt: 0.01


When using a gaussian pulse and a courant number of 1 we observe that the shape and the amplitude of the signal stays the same and we have a stable simulation.

In [7]:
draw_animation(pulse='gauss', courant=0.95)

Timestep dt: 0.0095


With a courant number of smaller than 1 the amplitude of the signal decays again and the gaussian widens due to the artificial diffusivity.

In [8]:
draw_animation(pulse='gauss', courant=1.05)

Timestep dt: 0.0105


With a courant number greater than 1 the simulation is unstable again.